In [8]:
import json
import os
import fnmatch
import pandas as pd
import re
import numpy as np

In [9]:
####### example to access text of json file


s1 = pd.Series([0,'test', 0])
df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
index = 0

for path, dirs, files in os.walk('twitter-english/'):
    if (path.endswith('source-tweet') or path.endswith('replies')):
        for filename in os.listdir(path):
            if filename.endswith('.json'):
                with open(os.path.join(path,filename)) as jsonFile:
                    jsonSourceTweet = json.load(jsonFile)

                    text_tweet = jsonSourceTweet['text']
                    id_tweet = int(re.sub("[^0-9]", "", filename))

                    newline = [id_tweet, text_tweet, index]
                    df.loc[index] = newline

                    index += 1




In [10]:
df.head()

,id,text,label
0,529661510369284096,"@mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...",0
1,529661727940431872,@halihamilton @mikeyerxa gonna try to make it ...,1
2,529661317087768576,@mikeyerxa you should send Prince a set of ten...,2
3,529666750330449920,"@NobleRobel Girl, don't be mean. Age ain't n...",3
4,529660296080916480,Prince is playing a secret show in Toronto ton...,4


In [11]:
df_label = pd.read_json('train-key.json')

In [12]:
df_label.columns = ['label', 'b']
df_label.head()

,label,b
501760642928635904,comment,NaN
500270212198174720,comment,NaN
524971210275565568,comment,NaN
552836882770690049,comment,NaN
500289931097296897,support,NaN


In [13]:
for i in range(len(df)):
    try:
        index = df.loc[i].id
        df.loc[i, 'label'] = df_label.loc[str(index)].label
    except KeyError:
        df.loc[i, 'label'] = 'not_found'


In [14]:
#save not found labels because they are the eval data
index_to_save = df[df.label == "not_found"].index
mask_save = df.index.isin(index_to_save)
df_eval = df[mask_save]
df_label_eval = pd.read_json('dev-key.json')
df_label_eval.columns = ['label', 'b']


for i in range(len(df_eval)):
    index = df_eval.iloc[i].id
    df_eval.loc[i, 'label'] = df_label_eval.loc[str(int(index))].label


<ipython-input-14-e2105947a041>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eval.loc[i, 'label'] = df_label_eval.loc[str(int(index))].label


KeyError: '524978844155912192'

In [15]:

type(df.id)

pandas.core.series.Series

In [16]:
#df_eval.to_csv('eval_data.csv')

In [17]:

#get rid of not found tag

index_to_delete = df[df.label == "not_found"].index
mask = np.logical_not(df.index.isin(index_to_delete))

df_final = df[mask]
df_final.label.value_counts()


comment    2907
support     910
query       358
deny        344
Name: label, dtype: int64

In [18]:
def create_data(path_training, path_answer):
    s1 = pd.Series([0,'test', 0])
    df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
    index = 0

    for path, dirs, files in os.walk(path_training):
        if (path.endswith('source-tweet') or path.endswith('replies')):
            for filename in os.listdir(path):
                if filename.endswith('.json'):
                    with open(os.path.join(path,filename)) as jsonFile:
                        jsonSourceTweet = json.load(jsonFile)

                        text_tweet = jsonSourceTweet['text']
                        id_tweet = int(re.sub("[^0-9]", "", filename))

                        newline = [id_tweet, text_tweet, index]
                        df.loc[index] = newline

                        index += 1

    #rename column into label                    
    df_label = pd.read_json(path_answer)
    first_col = df_label.columns[0]
    df_label.rename(columns={first_col:'label'}, inplace=True)
    
    #add labels
    for i in range(len(df)):
        try:
            index = df.loc[i].id
            df.loc[i, 'label'] = df_label.loc[str(index)].label
        except KeyError:
            df.loc[i, 'label'] = 'not_found'

    #delete unecessary labels
    index_to_delete = df[df.label == "not_found"].index
    mask = np.logical_not(df.index.isin(index_to_delete))
    df_final = df[mask]


    return df_final

#tes_data_csv = create_data('twitter-en-test-data', 'final-eval-key.json')


In [20]:
tweets = df.text

In [28]:
df.loc[9].text

"@davidleyes @EntCity: I'm crying as well... Don't you worry!"

In [35]:
tweets.str.contains('http') & tweets.str.contains('#')

0       False
1       False
2       False
3       False
4       False
        ...  
5563    False
5564    False
5565    False
5566    False
5567     True
Name: text, Length: 5568, dtype: bool

In [37]:
example_tweet = tweets.loc[5567]
example_tweet

"Was Vladimir Putin netralized by an internal coup? Or maybe he's vacationing in Harper's closet:  http://t.co/tlr3cFLEyv #LookingForNarnia"

In [38]:
#remove links
re.sub(r'http\S+', '', example_tweet)          

"Was Vladimir Putin netralized by an internal coup? Or maybe he's vacationing in Harper's closet:   #LookingForNarnia"

In [59]:

import string
def clean_data(text):
    #remove links
    text = re.sub(r'http\S+', '', text)

    #remove punct except @
    punc_to_remove = string.punctuation
    punc_to_remove.replace('@', '')
    text = "".join([char for char in text if char not in punc_to_remove])
          
    #lower case
    return text.lower()


In [63]:
df['text'] = df['text'].apply(lambda x: clean_data(x))

In [64]:
df.text

0       mikeyerxa excuse me wheres my invite to the la...
1       halihamilton mikeyerxa gonna try to make it to...
2       mikeyerxa you should send prince a set of tenn...
3       noblerobel girl dont be mean   age aint nothin...
4       prince is playing a secret show in toronto ton...
                              ...                        
5563    patondabak it is disconcerting that the man wi...
5564    patondabak or maybe he just needs a break from...
5565    patondabak maybe just maybe… hes huddling with...
5566    patondabak im going with neutralized and their...
5567    was vladimir putin netralized by an internal c...
Name: text, Length: 5568, dtype: object